In [ ]:
# libraries
import matplotlib.pyplot as plt
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import NiftiLabelsMasker
from nilearn import datasets
from nilearn import plotting
import nibabel as nib
import numpy as  np
import os
from numpy import loadtxt
import pyriemann
from scipy.stats import ks_2samp
import pandas
import sys

In [ ]:
# subj in each UIUC group
#TINN 14 subs = ['218','216','150','276','130','132','274','135','136','286','292','182','233','157']
#HLOS 16 subs = ['285','284','191','298','201','230','243','250','253','270','271','273','226','189','299','125']
#CTRL 18 subs = ['200','169','168','167','166','199','220','258','259','126','261','123','122','120','117','113','275','115']
#TNHL 37 subs = ['131','140','257','277','279','280','282','108','106','293','105','129','256','247','146','184','187','188','174','163','197','162','160','205','255','207','213','219','222','223','154','227','153','152','246','176','208']

In [ ]:
# downloading pearson fc matrices for UIUC data (2 scans from ses A and 2 from ses B)
dataset = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=2, data_dir=None, base_url=None, resume=True, verbose=1)
atlas_filename = dataset.maps
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True)
correlation_measure = ConnectivityMeasure(kind='correlation')
subs = ['205','255','207','213','219','222','223','154','227','153','152','246','176','208','102']
sess = ['A', 'B']

for sub in range(0,50):
    for ses in range(0,2):
        os.chdir('/data/UIUC/derivatives/fmriprep/sub-' + subs[sub] + '/ses-' + sess[ses] + '/func')
        if ses == 0:
            for pt in range(1,3):
                img = nib.load('sub-' + subs[sub] + '_ses-' + sess[ses] + '_task-rest_run-' + str(pt) + '_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
                if pt == 1:
                    run1 = masker.fit_transform(img)
                    correlation_matrix1 = correlation_measure.fit_transform([run1])[0]
                elif pt == 2:
                    run2 = masker.fit_transform(img)          
                    correlation_matrix2 = correlation_measure.fit_transform([run2])[0]
        elif ses == 1:
            for pt in range(1,3):
                img = nib.load('sub-' + subs[sub] + '_ses-' + sess[ses] + '_task-rest_run-' + str(pt) + '_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
                if pt == 1:
                    run1 = masker.fit_transform(img)
                    correlation_matrix3 = correlation_measure.fit_transform([run1])[0]
                elif pt == 2:
                    run2 = masker.fit_transform(img)          
                    correlation_matrix4 = correlation_measure.fit_transform([run2])[0]
    os.chdir('/data/hcp_working_folder/tdhore2/riemann2/fc/TNHL')
    np.savez('fc_' + subs[sub], Arun1 = correlation_matrix1, Arun2 = correlation_matrix2, Brun1 = correlation_matrix3, Brun2 = correlation_matrix4)

In [56]:
# riemann ingroup UIUC data
subs1 = ['131','140','257','277','279','280','282','108','106','293','105','129','256','247','146','184','187','188','174','163','197','162','160','205','255','207','213','219','222','223','154','227','153','152','246','176','208']
riemann_dist = np.zeros((4*37,4*37))
filenames = ['Arun1', 'Arun2', 'Brun1', 'Brun2']
os.chdir('/data/hcp_working_folder/tdhore2/riemann2/fc/TNHL')
for sub in range(0,37):
    sub1 = subs1[sub]
    npzfile1 = np.load('fc_' + sub1 + '.npz')
    for subb in range(0,37):
        sub2 = subs1[subb]
        npzfile2 = np.load('fc_' + sub2 + '.npz')
        for pt in range(0,4):
            for ptt in range(0,4):
                x = pyriemann.utils.distance.distance_riemann(npzfile1[filenames[pt]], npzfile2[filenames[ptt]])
                riemann_dist[4*sub+pt, 4*subb+ptt] = x
os.chdir('/data/hcp_working_folder/tdhore2/riemann2/')
np.savez('riemann_inngroup_TNHL', riemann = riemann_dist)

In [51]:
#riemann dist b/w groups UIUC data
subs1 = ['131','140','257','277','279','280','282','108','106','293','105','129','256','247','146','184','187','188','174','163','197','162','160','205','255','207','213','219','222','223','154','227','153','152','246','176','208']
subs2 = ['200','169','168','167','166','199','220','258','259','126','261','123','122','120','117','113','275','115']
riemann_dist = np.zeros((4*37,4*18))
filenames = ['Arun1', 'Arun2', 'Brun1', 'Brun2']
for sub in range(0,37):
    os.chdir('/data/hcp_working_folder/tdhore2/riemann2/fc/TNHL')
    sub1 = subs1[sub]
    npzfile1 = np.load('fc_' + sub1 + '.npz')
    for subb in range(0,18):
        os.chdir('/data/hcp_working_folder/tdhore2/riemann2/fc/CTRL')
        sub2 = subs2[subb]
        npzfile2 = np.load('fc_' + sub2 + '.npz')
        for pt in range(0,4):
            for ptt in range(0,4):
                x = pyriemann.utils.distance.distance_riemann(npzfile1[filenames[pt]], npzfile2[filenames[ptt]])
                riemann_dist[4*sub+pt, 4*subb+ptt] = x
os.chdir('/data/hcp_working_folder/tdhore2/riemann2/')
np.savez('riemann_betweengroup_TNHL_CTRL', riemann = riemann_dist)

In [17]:
# KS test
os.chdir('/data/hcp_working_folder/tdhore2/riemann2')
npzfile1 = np.load('riemann_betweengroup_HCP_CTRL.npz')
run1 = npzfile1['riemann']
run1 = run1.flatten()
os.chdir('/data/hcp_working_folder/tdhore2/')
npzfile2 = np.load('riemann_inngroup_HCP.npz')
run2 = npzfile2['riemann']
run2 = run2.flatten()
ks_2samp(run1, run2)

KstestResult(statistic=0.6128444444444444, pvalue=1.0)

In [ ]:
# downloading pearson fc matrices for HCP data (4 scans/subj)
dataset = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=2, data_dir=None, base_url=None, resume=True, verbose=1)
atlas_filename = dataset.maps
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True)
subs = ['469961','475855','479762','480141','481042','481951','485757','486759','495255','497865','499566','500222','506234','510225','510326','512835','513130','513736','516742','517239','518746','519647','519950','520228','522434','523032','525541','529549','529953','530635','531536','531940','536647','541640','541943','545345','547046','548250','552241','552544','553344','555348','555651','555954','557857','558657','558960','559457','561242','561444']
files = ['REST1_LR', 'REST1_RL', 'REST2_LR', "REST2_RL"]
correlation_measure = ConnectivityMeasure(kind='correlation')

for sub in range(0,50):
    os.chdir('/data/hcp_working_folder/tdhore2/HCP/subj/' + subs[sub] + '/func')

    for pt in range(0,4):
        img = nib.load('rfMRI_' + files[pt] + '.nii.gz')
        if pt == 0:
            LR1 = masker.fit_transform(img)
            correlation_matrix1 = correlation_measure.fit_transform([LR1])[0]
        elif pt == 1:
            RL1 = masker.fit_transform(img)          
            correlation_matrix2 = correlation_measure.fit_transform([RL1])[0]
        elif pt == 2:
            LR2 = masker.fit_transform(img)
            correlation_matrix3 = correlation_measure.fit_transform([LR2])[0]
        else: 
            RL2 = masker.fit_transform(img)
            correlation_matrix4 = correlation_measure.fit_transform([RL2])[0]
    os.chdir('/data/hcp_working_folder/tdhore2/HCP/fc')
    np.savez('fc_' + subs[sub], LR1 = correlation_matrix1, RL1 = correlation_matrix2, LR2 = correlation_matrix3, RL2 = correlation_matrix4)

In [10]:
# HCP riemann ingroup
subs1 = ['469961','475855','479762','480141','481042','481951','485757','486759','495255','497865','499566','500222','506234','510225','510326','512835','513130','513736','516742','517239','518746','519647','519950','520228','522434']
riemann_dist = np.zeros((4*25,4*25))
filenames = ['LR1', 'LR2', 'RL1', 'RL2']
os.chdir('/data/hcp_working_folder/tdhore2/HCP/fc')
for sub in range(0,25):
    sub1 = subs1[sub]
    npzfile1 = np.load('fc_' + sub1 + '.npz')
    for subb in range(0,25):
        sub2 = subs1[subb]
        npzfile2 = np.load('fc_' + sub2 + '.npz')
        for pt in range(0,4):
            for ptt in range(0,4):
                x = pyriemann.utils.distance.distance_riemann(npzfile1[filenames[pt]], npzfile2[filenames[ptt]])
                riemann_dist[4*sub+pt, 4*subb+ptt] = x
os.chdir('/data/hcp_working_folder/tdhore2')
np.savez('riemann_inngroup_HCP', riemann = riemann_dist)

In [ ]:
# HCP+UIUC riemann b/w group
subs1 = ['131','140','257','277','279','280','282','108','106','293','105','129','256','247','146','184','187','188','174','163','197','162','160','205','255','207','213','219','222','223','154','227','153','152','246','176','208']
subs2 = ['106319','123117','211922','178142','562446','704238','180230','168240','129331','419239','154330','137128','559053','106824','356948','212823','151425','289555','103111','119126','192136','524135','100307','765056','102109']
riemann_dist = np.zeros((4*37,4*25))
filenames1 = ['Arun1', 'Arun2', 'Brun1', 'Brun2']
filenames2 = ['LR1', 'LR2', 'RL1', 'RL2']
for sub in range(0,37):
    os.chdir('/data/hcp_working_folder/tdhore2/riemann2/fc/TNHL')
    sub1 = subs1[sub]
    npzfile1 = np.load('fc_' + sub1 + '.npz')
    for subb in range(0,25):
        os.chdir('/data/hcp_working_folder/tdhore2/riemann2/HCP_fc')
        sub2 = subs2[subb]
        npzfile2 = np.load('fc_' + sub2 + '.npz')
        for pt in range(0,4):
            for ptt in range(0,4):
                x = pyriemann.utils.distance.distance_riemann(npzfile1[filenames1[pt]], npzfile2[filenames2[ptt]])
                riemann_dist[4*sub+pt, 4*subb+ptt] = x
os.chdir('/data/hcp_working_folder/tdhore2/riemann2')
np.savez('riemann_betweengroup_HCP_TNHL', riemann = riemann_dist)

In [16]:
# HCP+UIUC riemann b/w group
subs1 = ['469961','475855','479762','480141','481042','481951','485757','486759','495255','497865','499566','500222','506234','510225','510326','512835','513130','513736','516742','517239','518746','519647','519950','520228','522434']
subs2 = ['200','169','168','167','166','199','220','258','259','126','261','123','122','120','117','113','275','115']
riemann_dist = np.zeros((4*25,4*25))
filenames1 = ['Arun1', 'Arun2', 'Brun1', 'Brun2']
filenames2 = ['LR1', 'LR2', 'RL1', 'RL2']
for sub in range(0,25):
    os.chdir('/data/hcp_working_folder/tdhore2/HCP/fc')
    sub1 = subs1[sub]
    npzfile1 = np.load('fc_' + sub1 + '.npz')
    for subb in range(0,18):
        os.chdir('/data/hcp_working_folder/tdhore2/riemann2/fc/CTRL')
        sub2 = subs2[subb]
        npzfile2 = np.load('fc_' + sub2 + '.npz')
        for pt in range(0,4):
            for ptt in range(0,4):
                x = pyriemann.utils.distance.distance_riemann(npzfile1[filenames2[pt]], npzfile2[filenames1[ptt]])
                riemann_dist[4*sub+pt, 4*subb+ptt] = x
os.chdir('/data/hcp_working_folder/tdhore2')
np.savez('riemann_betweengroup_HCP_CTRL', riemann = riemann_dist)